# Latencia
Tiempo que tarda un modelo en procesar una entrada y generar una prediccion

In [1]:
import os, time
import importlib , latency, pandas as pd # Importar importlib para recargar módulos
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), '..', '.env')
importlib.reload(latency)
from glob import glob
from latency import Latency

# Configuracion
detector_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('VITPOSE_SUBPATH'), os.getenv('DETECTOR_SUBPATH'))
estimator_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('VITPOSE_SUBPATH'), os.getenv('POSE_SUBPATH'))
images_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('IMAGES_SUBPATH'))
labels_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('LABELS_SUBPATH'))
videos_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('VIDEOS_SUBPATH'))
evaluator = Latency(os.getenv('BASE_PATH'), images_path, labels_path, videos_path, detector_path, estimator_path)
thresholds = [0.1, 0.5, 0.9]

/Users/apple/Documents/pose_estimation/ViTPosev4.57/vitpose_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargando modelos en el dispositivo: cpu


`use_fast` is set to `True` but the image processor class does not have a fast version.  Falling back to the slow version.


# Calculo de latencia en un video especifico

In [2]:
# Obtener la lista de todas las imágenes en la carpeta
video = 'ejercicio01 - frente.mp4'
video_path = os.path.join(videos_path, 'uziel', 'ejercicio01', 'conLuz', video)
print(video_path)

results = []
for threshold in thresholds:
    results = evaluator.evaluate_video(
        videos_path=video_path, 
        threshold=threshold,
        results=results,
        frame_skip=5,
    )

/Users/apple/Documents/pose_estimation/dataset/videos/uziel/ejercicio01/conLuz/ejercicio01 - frente.mp4


Procesando ejercicio01 - frente.mp4 (th:0.9): 100%|██████████| 35/35 [00:43<00:00,  1.23s/it]


# Calculo de latencia en videos dentro de una carpeta

In [4]:

folder_path = os.path.join(videos_path, 'uziel')
video_extensions = ['*.mp4', '*.avi', '*.mov']
video_files = []
results = []

for ext in video_extensions:
    video_files.extend(glob(os.path.join(folder_path, '**', ext), recursive=True))

for video_path in video_files:
    for threshold in thresholds:
        results = evaluator.evaluate_video(
            videos_path=video_path, 
            threshold=threshold, 
            results=results,
            frame_skip=5,
        )

Procesando ejercicio02 - izquierda obstaculo.mp4 (th:0.1):  12%|█▏        | 5/43 [00:05<00:45,  1.20s/it]

KeyboardInterrupt: 

# Generando XLS de los resultados de los videos

In [3]:
df_results = pd.DataFrame(results)
print(df_results.head().to_string)
output_excel_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('VITPOSE_SUBPATH'), 'velocidad', 'latency', 'vitpose_video_latency_results.xlsx')
df_results.to_excel(output_excel_path, index=False, engine='openpyxl')

print(f"Los resultados se han guardado en {output_excel_path}")

<bound method DataFrame.to_string of                       video video_resolution  threshold  video_fps  \
0  ejercicio01 - frente.mp4          480x864        0.1  30.016466   
1  ejercicio01 - frente.mp4          480x864        0.1  30.016466   
2  ejercicio01 - frente.mp4          480x864        0.1  30.016466   
3  ejercicio01 - frente.mp4          480x864        0.1  30.016466   
4  ejercicio01 - frente.mp4          480x864        0.1  30.016466   

   frame_skip   latency_ms  processed_frames  total_frames_in_file  
0           5  1331.894159                 1                   175  
1           5  1100.665808                 2                   175  
2           5  1362.099886                 3                   175  
3           5  1194.923878                 4                   175  
4           5  1109.618902                 5                   175  >
Los resultados se han guardado en /Users/apple/Documents/pose_estimation/ViTPosev4.57/velocidad/latency/vitpose_video_latency_r